In [1]:
print("hello chatbot")

hello chatbot


In [2]:
%pwd

'/Users/pankajkumar/Downloads/Coding/Projects/Medical-Chatbot-with-LLMs-LangChain-Pinecone-React-AWS/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/pankajkumar/Downloads/Coding/Projects/Medical-Chatbot-with-LLMs-LangChain-Pinecone-React-AWS'

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # For splitting text into smaller chunks

/Users/pankajkumar/.local/share/virtualenvs/Medical-Chatbot-with-LLMs-LangChain-Pinecone-React-AWS-akr9m8/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Extract text from PDF files in the 'data' directory
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_files("data")

In [ ]:
extracted_data

In [10]:
len(extracted_data)

637

In [3]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [12]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [ ]:
minimal_docs

In [14]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [15]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 5859


In [ ]:
texts_chunk

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

/var/folders/4c/vdjg0l7x24b884_dnprm8xt40000gn/T/ipykernel_3024/171795349.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [18]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447728604078293,
 0.031023193150758743,
 0.006734972819685936,
 0.026109015569090843,
 -0.03936203196644783,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441446952521801,
 -0.047450508922338486,
 0.014758873730897903,
 0.07087536156177521,
 0.05552757531404495,
 0.019193297252058983,
 -0.026251330971717834,
 -0.01010951865464449,
 -0.026940520852804184,
 0.02230742573738098,
 -0.02222665771842003,
 -0.14969265460968018,
 -0.017493057996034622,
 0.007676220964640379,
 0.054352302104234695,
 0.0032544543500989676,
 0.03172598406672478,
 -0.08462141454219818,
 -0.029405983164906502,
 0.05159566551446915,
 0.04812401905655861,
 -0.0033147598151117563,
 -0.05827919393777847,
 0.04196927696466446,
 0.022210730239748955,
 0.128188818693161,
 -0.022338904440402985,
 -0.011656290851533413,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122607111930847,
 -0.031175294890999794,
 0.05269956961274147,
 0.04703482985496521,
 -0.08420304954051971,
 -0.030056187883019447,
 -0.02074483

In [20]:
print( "Vector length:", len(vector))

Vector length: 384


In [6]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [27]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [8]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [9]:
pc

In [10]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [27]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [11]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

# Add more data to the existing Pinecone index

In [32]:
dswith = Document(
    page_content="pankaj is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

docsearch.add_documents(documents=[dswith])

['2a501baa-76a1-411c-ab85-fe99e99dfeea']

In [13]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [14]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='2f3a524a-5884-4f97-851c-46501899fd1d', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='84552834-5ba8-424b-9d03-4735c93fa2c6', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='6e612ab4-9b46-420b-8dec-978c69f19460', metadata={'source': 'data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with 

In [25]:
from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = "sk-o"  # your OpenRouter key
# chatModel = ChatOpenAI(model="gpt-4o")
chatModel = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",  # 👈 important
    model="openai/gpt-oss-20b:free"
)

In [26]:
response = chatModel.invoke("What is Acromegaly and gigantism?")
print(response.content)

## Acromegaly vs. Gigantism  
*(Why the same hormone can make two very different appearances)*  

| **Feature** | **Acromegaly** | **Gigantism** |
|-------------|----------------|--------------|
| **Who is affected?** | Adults who have already finished bone growth (after epiphyseal closure) – usually late‑20s to early‑40s. | Children or adolescents who still have open growth plates. |
| **Primary symptom** | Symmetric enlargement of the parts of the skeleton and soft tissue that change after puberty – hands, feet, jaw, nose. | Excessive linear growth – the child becomes dramatically taller than peers. |
| **Bone involvement** | Mainly metaphyses & epiphyses that have already fused; 1st‑pass effects are limited to bone length, 2nd‑pass growth of cartilage. | Growth plates are still open, so GH stimulates continued longitudinal bone growth → tall stature. |
| **Typical body‑mass changes** | Weight gain, coarse facial features, hypertension, diabetes, arthropathy. | BMI similar to peers, 

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [18]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [28]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [30]:
# response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
response = rag_chain.invoke({"input": "what is pankaj"})
print(response["answer"])

Pankaj is a YouTube channel that offers tutorials on a variety of topics. It provides instructional videos aimed at helping viewers learn new skills. The channel covers subjects across many fields, similar to other educational YouTube channels.
